In this notebook we tried to build an LDA model using sklearn package. 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as plt
import seaborn as sns
import sklearn

In [2]:
# This block read dataset and look at its sample

df = pd.read_csv('../data/nyt_metadata_cleaned.csv')
df.sample(10)

,Unnamed: 0.1,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,...,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name,date
742,786,53947,there a resurgence of interest in the pioneeri...,https://www.nytimes.com/2023/05/06/nyregion/co...,There’s a resurgence of interest in the pionee...,connie converse was a pioneer of what become k...,MB,10.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Metropolitan,New York,"{'original': 'By Howard Fishman', 'person': [{...",News,nyt://article/513ca058-6f8f-5591-9154-77854a15...,1605.0,nyt://article/513ca058-6f8f-5591-9154-77854a15...,NaN,2023-05-06 09:00:21+00:00
3377,3532,56693,a hong kong judge postponed deciding whether t...,https://www.nytimes.com/2023/12/03/business/ch...,A Hong Kong judge postponed deciding whether t...,once china most prolific property developer ch...,B,1.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Business,Business Day,{'original': 'By Alexandra Stevenson and Tiffa...,News,nyt://article/b858f075-08ed-56bc-b358-33dc0274...,797.0,nyt://article/b858f075-08ed-56bc-b358-33dc0274...,NaN,2023-12-04 03:57:03+00:00
5371,5614,58775,workers had already begun dismantling the stat...,https://www.nytimes.com/2023/12/19/us/confeder...,Workers had already begun dismantling the stat...,a federal judge on tuesday cleared the way for...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Express,U.S.,"{'original': 'By Aimee Ortiz', 'person': [{'fi...",News,nyt://article/2387466f-82a3-5727-b97d-405b06d6...,689.0,nyt://article/2387466f-82a3-5727-b97d-405b06d6...,NaN,2023-12-20 03:58:48+00:00
4424,4625,57786,the ecologist karen lips observed frogs for se...,https://www.nytimes.com/video/opinion/10000000...,The ecologist Karen Lips observed frogs for se...,the ecologist karen lips observed frogs for se...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,multimedia,NaN,Opinion,"{'original': 'By Volker Schlecht, Alexander La...",Video,nyt://video/31c23fe9-349f-5a24-80b2-2aad345e37d6,0.0,nyt://video/31c23fe9-349f-5a24-80b2-2aad345e37d6,NaN,2023-12-12 09:55:00+00:00
2175,2286,55447,a couple can t agree on who gets to hang their...,https://www.nytimes.com/2023/05/18/magazine/ju...,A couple can’t agree on who gets to hang their...,austin writes i believe a coatrack should rema...,MM,20.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Magazine,Magazine,"{'original': 'By John Hodgman', 'person': [{'f...",News,nyt://article/f28d58dc-11cd-534b-9e0d-3063a836...,165.0,nyt://article/f28d58dc-11cd-534b-9e0d-3063a836...,NaN,2023-05-18 09:00:07+00:00
5836,6102,59263,the ukrainian authorities said they had uncove...,https://www.nytimes.com/2023/12/23/world/europ...,The Ukrainian authorities said they had uncove...,the ukrainian police have arrested a senior de...,A,11.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Foreign,World,"{'original': 'By Constant Méheut', 'person': [...",News,nyt://article/0b1bb200-26cb-5c67-a045-cdf74fea...,721.0,nyt://article/0b1bb200-26cb-5c67-a045-cdf74fea...,Europe,2023-12-23 12:55:31+00:00
3504,3668,56829,on the outskirts of mexico city biologists are...,https://www.nytimes.com/2023/12/05/science/mex...,"On the outskirts of Mexico City, biologists ar...",xochimilco is a large semirural district in th...,D,1.0,The New York Times,[],...,article,Science,Science,{'original': 'By Jennie Erin Smith and Luis An...,News,nyt://article/b0c917af-969f-5bdc-af2f-4f7c3d25...,2451.0,nyt://article/b0c917af-969f-5bdc-af2f-4f7c3d25...,NaN,2023-12-05 08:01:23+00:00
3208,3354,56515,maximilian davis ferragamo creative director s...,https://www.nytimes.com/2023/12/01/t-magazine/...,"Maximilian Davis, Ferragamo’s creative directo...",less than a week after presenting his third co...,M2,92

In [3]:
# For the test purpose, we use only abstract here.

X = df.abstract

X.sample(10)

6283    a lot of big movies by big name directors are ...
5608    the report from the network contagion research...
4461    king misuzulu claim to the throne in a storied...
5312    wildlife officials have released a small group...
1389    readers discuss whether the network should hav...
4559    ukraine military said that it had shot down al...
107     russian forces were reportedly moving into def...
3726    the justice department also said it had begun ...
1153    at the italian open women will compete for les...
1713    morant the star memphis grizzlies guard was fi...
Name: abstract, dtype: object

## Tokenization and Further Data preparing

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [5]:
# We start by tokenize the texts

print("Extracting features from the training data using a Count vectorizer")

vectorizer_count = CountVectorizer(#sublinear_tf=True, 
                                   #max_df=0.95, #remove words that have a document frequency strictly higher than 15%
                                   analyzer='word',
                                   min_df=10, #remove words that have a document frequency strictly lower than 0.1%
                                   stop_words='english', #removing stop words
                                   lowercase=True,
                                   ngram_range=(1,2), #unigrams and bigrams are generated
                                   token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                                   max_features=2**14)

#Now We train the vectorizer 
X_count = vectorizer_count.fit_transform(X)



Extracting features from the training data using a Count vectorizer


In [7]:
# Countvectorizor returns a dense matrix. Below we calculate the sparsicity of this model.
# Materialize the sparse data
X_dense = X_count.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((X_dense > 0).sum()/X_dense.size)*100, "%")

Sparsicity:  0.40648607508703466 %


## Model Optimized

In this part we use GridSearchCV to find the best number of topics as well as learning rate.

In [8]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [10]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30,35,40], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(X_count)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [10, 15, 20, 25, 30, 35, 40]})

In [12]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(X_count))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -87000.10962360006
Model Perplexity:  1624.4050019802207


## Visualization

Below we visualize some aspect of this dataset using the best lda model we got.

In [13]:
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.lda_model
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(best_lda_model, X_count, vectorizer_count, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2     -67.666817 -22.086937       1        1  11.123056
9     -35.298870 -85.348198       2        1  10.898281
0       8.898300  67.545334       3        1  10.876192
3     -11.562670 -36.900669       4        1  10.741442
8      64.068054  37.181404       5        1  10.524447
7      33.604126 -75.158455       6        1  10.011818
4     -12.175128  16.336332       7        1   9.395124
5      83.672836 -32.807758       8        1   9.045183
6     -60.065620  42.480133       9        1   8.900646
1      34.243195  -9.774357      10        1   8.483811, topic_info=           Term        Freq       Total Category  logprob  loglift
1086        new  753.000000  753.000000  Default  30.0000  30.0000
1798       york  257.000000  257.000000  Default  29.0000  29.0000
1094   new york  255.000000  255.000000  Default  28.0000  28.0000
1670      trump  192.000000  192.000000  Default  27.0000  27.0000
1229  president  338.000000  338.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1181     people   35.883420  237.843835  Topic10  -4.7414   0.5757
145       biden   27.179249  131.428901  Topic10  -5.0192   0.8910
1783      world   27.544582  172.595653  Topic10  -5.0059   0.6319
648        free   20.729361   40.898450  Topic10  -5.2901   1.7875
8       accused   19.766495   51.595522  Topic10  -5.3377   1.5076

[581 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2         5  0.056493      abortion
2        10  0.932128      abortion
4         6  0.042475        access
4         8  0.084950        access
4        10  0.849499        access
...     ...       ...           ...
1802      3  0.201495         young
1802      6  0.688441         young
1802      8  0.016791         young
1802     10  0.083956         young
1803      6  0.960869  young people

[1325 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 1, 4, 9, 8, 5, 6, 7, 2])

## Recommendation

Below we recommend articles based on a given article. We need to first vectorize it, then transform it using our model. We then give the class it belongs to, and recommend similar articles based on cosine_similarity

In [34]:
feature_names = vectorizer_count.get_feature_names_out()
best_n_components = model.best_params_.get("n_components")
best_learning_rates = model.best_params_.get("learning_rates")

In [45]:
def show_topics(vectorizer=vectorizer_count, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer_count, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,years,president,make,house,new,right,biden,life,debt,things,love,music,work,president biden,government
Topic 1,state,said,president,south,rights,law,people,military,abortion,say,administration,governor,texas,palestinian,world
Topic 2,new,york,new york,city,states,times,united,united states,york times,york city,pandemic,migrants,expected,mayor,world
Topic 3,israel,gaza,said,hamas,war,israeli,world,time,intelligence,need,country,military,change,artificial,family
Topic 4,ukraine,use,past,russia,year,war,articles,ukrainian,minister,prime,government,sentence,word,bank,past year
Topic 5,new,readers,university,students,health,times,news,week,time,debate,college,los,angeles,los angeles,art
Topic 6,year,said,old,appeared,people,latest,print,corrections,corrections appeared,appeared print,end,year old,killed,friday,left
Topic 7,new,including,story,books,like,women,people,don,game,young,novel,tell,author,real,season
Topic 8,trump,president,court,donald,donald trump,federal,case,election,biden,supreme,voters,supreme court,white,trial,house
Topic 9,year,said,police,social,best,state,new,media,social media,team,king,britain,desantis,prosecutors,movie


In [51]:
# Define function to predict topic for a given text document.
#nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text):
    # Vectorize transform
    mytext = vectorizer_count.transform(text)

    # LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    print()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["Result of Trump trial"]
topic, prob_scores = predict_topic(text = mytext)


[[0.025      0.02500065 0.02500025 0.025      0.025      0.025
  0.02500407 0.025      0.77499503 0.02500001]]


In [52]:
from sklearn.metrics.pairwise import cosine_similarity

X_transform = best_lda_model.transform(X_count)

# Calculate the cosine similarity of articles
X_sim = cosine_similarity(X_transform)



array([0.39326744, 0.00909108, 0.0090917 , 0.00909109, 0.11531775,
       0.00909176, 0.00909129, 0.00909109, 0.42777379, 0.00909301])

In [56]:
# Randomly choose an article:

art_ind = X.sample().index

# Find the corresponding similarities
sim_art = X_sim[art_ind][0]


sorted_sim_art_index = np.flip(sim_art.argsort()[:-1])
sorted_sim_art_index[:5]

#sim_art[sorted_sim_art_index[:5]]
print(X[art_ind])

print("Topics of this article are:")
topic, prob_scores = predict_topic(text = X[art_ind])
print(topic)
print("Recommended articles are:")
#print("Recommended articles:")
print(X.iloc[sorted_sim_art_index[:5]])

2693    readers respond to kate pickert essay about th...
Name: abstract, dtype: object
Topics of this article are:
['new', 'readers', 'university', 'students', 'health', 'times', 'news', 'week', 'time', 'debate', 'college', 'los', 'angeles', 'los angeles', 'art']
Recommended articles are:
3276    she is essentially one on the new film but she...
2879    we asked clinicians and researchers to share t...
606     sweating stomach pain headaches experts explai...
2845    there are healthy ways to have uncomfortable c...
1159    americans have been dealing with rapid inflati...
Name: abstract, dtype: object


## Conclusion
We use gridCVsearch to find the best topic numbers and learing rates for lda model. Given any articles in the dataset, we could report its topic and recommend some similar articles.


## Reference

[Grid Search](https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/)
[Topic Visualize](https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html)